# Evaluation

In [13]:
import os
import sys

sys.path.append(os.getcwd())

from clustpy.deep.autoencoders import FeedforwardAutoencoder



ModuleNotFoundError: No module named 'practical.DeepClustering.DeepECT.evaluation'

## Flat Clustering

### MNIST

In [9]:
evaluation.evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=evaluation.DatasetType.MNIST, seed=42
)

NameError: name 'evaluation' is not defined

### USPS

### Fashion-MNIST

### Reuters

## Hierarchical Clustering

### MNIST

### USPS

### Fashion-MNIST

### Reuters